# [ICTCOG AI Academy] 7기 고급시각저녁반
# Review
### AI Index Report 2021
#### Computer Vision 방향
  - Resnet이 인간을 뛰어넘었기 때문에 이제는 경령화
- 퍼포먼스는 고원현상에 도달함
- GPU, 분산처리를 통해 성능 개선, 학습 시간, cost 축소


### Review
#### 1. 이미지 데이터 Load 

In [ ]:
import tensorflow as tf

In [ ]:
#tensorflow에서 제공하는 최고의 performance :tf.data.Dataset, tf.function
tf.keras.preprocessing.image_dataset_from_directory
tf.keras.preprocessing.image.ImageDataGenerator
tf.io.read_file
tf.io.decode_jpeg #여러가지

#### 단점
- 범용성이 Numpy 보다 떨어짐
  - 다양한 프레임워크 사용하기에 복잡함
- 난이도
  - 핑수적 기능 외에 추가 기능이 비교적 적어서 직접 만듦어야함

기타 Numpy 호환 Library
- imageio
- Scipy
- scikit-image
- opencv
- PIL
  - Tensorflow, PyTorch에서 내부적으로 사용

### 2. 모델 
- Functional programming + OOP(객체지향)

1. tf.nn 
  - from scartch
2. tf.estimator
  - greaident based model
3. tf.keras.models.Sequential
  - 단방향
4. tf.keras.models.Model
  - multi-inputs, multi-outputs
5. tf.keras.models.Model 상속
  - 제공하지 않는 모델 만들때 사용

> 모델 구성에는 답이 없음
  - "...안타깝지만 어떤 모델의 (층의 개수나 뉴런 개수에 해당하는) 적절한 크기나 구조를 결정하는 마법같은 공식은 없습니다. "

### 모델
- 논문 직접 구현
- 이미 만들어진 모델 가져다가 사용
  - Model Zoo
  - Awsome ___
  - Paperwithcode(sota)
  - tensorflow addons
#### Transfer Learining
- feature extractor, fine-tuning
  - 유사도와 데이터 양에 따라서 형태 결정
  - 모델의 일부를 가져다 쓰는 경우 많았음

- customizing 필요 
 - learning technique
 - compile 의 loss, optimizer 
  - 기존에 만들어진 것 사용하거나 직접 구현 필요

In [ ]:
model.compile( 
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],)

### 3.성능 높이기
### Data Augmentation
- 기존의 데이터를 이용해서 가짜 데이터 생성
  - jittering( random cropping , flipping, rotating)
- 모델 성능향상보다는 overfitting 방지용

### 4.Learning technique 
- fit
- fit_generator
- train_on_batch
- **tf.GradientTape**
  - with 구문과 사용
  - 직접 구현
- train from estimator


---
### A.Obejct Detection
- 사람을 뛰어넘음
#### 1 stage
- YOLO v1,v2,v3,v4,v5
  > 참고 :https://pjreddie.com/darknet/yolo/
- 빠른 속도
- 활용: real-time detection

#### 2 stage
- R-CNN, SPPnet, Fast R-CNN, Faster R-CNN
- 높은 성능 
- 활용: Data Annotation(Object Detection)


### B.Segmentation

#### Semantic Segmentation, Instance Segmentation
- 픽셀 단위로 recognition
- 같은 픽셀끼리 합침


### C.Generation
- AutoEncoder, VAE, **GAN** 활용
> - 활용 테크닉 leaky relu, 다양한 normalization 
- VAE  
- GAN  흐릿함


> python 2에서 3  버전 호환이 되지 않아서 six library 사용 
- segmentation, detection 은 이미지와 annotation 파일까지 필요함
  - annotation tool도 필요함
  - 기존의것 재학습 하는 경우가 많음 
- 참고 : https://github.com/tensorflow/hub/blob/master/examples/colab/tf2_object_detection.ipynb 

- 컨볼루션 사용해서 크기를 줄이고 원본 영역을 찾아야하므로 원래 크기로 복원 
  - feature extraction + generation 기법 

#### [Segmentation](https://www.tensorflow.org/tutorials/images/segmentation?hl=ko)
U-net
- 컨볼루션을 통해 잃어버린 detail을 skip connection으로 받아옴
- 복원 시 더 잘 복원됨


- pipeline 구성 시 효율적 


In [3]:
!pip install -q git+https://github.com/tensorflow/examples.git
!pip install -q -U tfds-nightly

     |████████████████████████████████| 3.8MB 4.5MB/s 


In [4]:
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [5]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [6]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [7]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [14]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
#map 적용
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)
# 최적화
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

### U-net
- MobileNetV2 Transfer Learning

In [8]:
OUTPUT_CHANNELS = 3
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

#이 층들의 활성화를 이용합시다
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# 특징추출 모델을 만듭시다
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [9]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [12]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # 다운샘플링
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])  #reverse 해서 순서 바꿈 

  # skip connection , 업샘플링ef unet_model(output_channels):
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # 이 모델의 마지막 층입니다
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [13]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #numerical stability 
              metrics=['accuracy'])

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\n에포크 이후 예측 예시 {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

functional paradigm  , OOP 적용 되어 있음
### 복습
1. 데이터를 불러오기
2. 이미지 데이터 EDA
3. normalization
  - [0,1] , [-1,1] 
4. 모델 구조 정하기
  - 모델 상속 
  - data augmentation 은 먼저 학습해보고 overfitting 생기면 적용
- Tensorflow 하드웨어 지원 최대화 가능

참고 : https://www.tensorflow.org/tutorials/quickstart/advanced?hl=ko

In [ ]:
class MyModel(Model): #상속
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:  #gradient tape 
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))